In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from pyvis import network as net
import random
%matplotlib inline

In [45]:
data = pd.read_csv("eu-car-trade.csv")
data["reporter"] = data["reporter"].astype(str)
data["partner"] = data["partner"].astype(str)
data.dtypes

DATAFLOW        object
LAST UPDATE     object
freq            object
reporter        object
partner         object
product          int64
flow             int64
indicators      object
TIME_PERIOD      int64
OBS_VALUE        int64
OBS_FLAG       float64
dtype: object

In [46]:
data = data[data["TIME_PERIOD"] == 2022] # Set year that we consider

In [47]:
filtered_data = data[data["OBS_VALUE"] > 200000000] # Only consider trade values over 200 million Euros
filtered_data = filtered_data[filtered_data['flow'] == 2] # 1 - imports, 2 - exports
filtered_data = filtered_data.reset_index()
filtered_data

,index,DATAFLOW,LAST UPDATE,freq,reporter,partner,product,flow,indicators,TIME_PERIOD,OBS_VALUE,OBS_FLAG
0,66,ESTAT:DS-018995(1.0),15/11/23 11:00:00,A,AT,BE,781,2,VALUE_IN_EUROS,2022,433252066,NaN
1,148,ESTAT:DS-018995(1.0),15/11/23 11:00:00,A,AT,CN,781,2,VALUE_IN_EUROS,2022,650758147,NaN
2,170,ESTAT:DS-018995(1.0),15/11/23 11:00:00,A,AT,CZ,781,2,VALUE_IN_EUROS,2022,454697203,NaN
3,178,ESTAT:DS-018995(1.0),15/11/23 11:00:00,A,AT,DE,781,2,VALUE_IN_EUROS,2022,1299332737,NaN
4,248,ESTAT:DS-018995(1.0),15/11/23 11:00:00,A,AT,GB,781,2,VALUE_IN_EUROS,2022,503054626,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
242,16633,ESTAT:DS-018995(1.0),15/11/23 11:00:00,A,SK,IT,781,2,VALUE_IN_EUROS,2022,1031413951,NaN
243,16805,ESTAT:DS-018995(1.0),15/11/23 11:00:00,A,SK,NL,781,2,VALUE_IN_EUROS,2022,394404072,NaN
244,16846,ESTAT:DS-018995(1.0),15/11/23 11:00:00,A,SK,PL,781,2,VALUE_IN_EUROS,2022,668597863,NaN
245,16900,ESTAT:DS-018995(1.0),15/11/23 11:00:00,A,SK,SE,781,2,VALUE_IN_EUROS,2022,444056747,NaN


In [13]:
data["reporter"].unique()

array(['AT', 'BE', 'BG', 'CY', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR',
       'GR', 'HR', 'HU', 'IE', 'IT', 'LT', 'LU', 'LV', 'MT', 'NL', 'PL',
       'PT', 'RO', 'SE', 'SI', 'SK'], dtype=object)

In [14]:
codes = pd.read_csv("country-codes.csv") # Get country codes
codes.loc[codes["Code"] == "RS", "Code"] = "XS" # Correct Serbia's country code

countries = dict(zip(codes['Name'], codes['Code']))

countries_by_code = {v: k for k, v in countries.items()} # Invert country code dictionary to have the codes as the keys
countries_by_code

{'AF': 'Afghanistan',
 'AX': 'Åland Islands',
 'AL': 'Albania',
 'DZ': 'Algeria',
 'AS': 'American Samoa',
 'AD': 'Andorra',
 'AO': 'Angola',
 'AI': 'Anguilla',
 'AQ': 'Antarctica',
 'AG': 'Antigua and Barbuda',
 'AR': 'Argentina',
 'AM': 'Armenia',
 'AW': 'Aruba',
 'AU': 'Australia',
 'AT': 'Austria',
 'AZ': 'Azerbaijan',
 'BS': 'Bahamas',
 'BH': 'Bahrain',
 'BD': 'Bangladesh',
 'BB': 'Barbados',
 'BY': 'Belarus',
 'BE': 'Belgium',
 'BZ': 'Belize',
 'BJ': 'Benin',
 'BM': 'Bermuda',
 'BT': 'Bhutan',
 'BO': 'Bolivia, Plurinational State of',
 'BQ': 'Bonaire, Sint Eustatius and Saba',
 'BA': 'Bosnia and Herzegovina',
 'BW': 'Botswana',
 'BV': 'Bouvet Island',
 'BR': 'Brazil',
 'IO': 'British Indian Ocean Territory',
 'BN': 'Brunei Darussalam',
 'BG': 'Bulgaria',
 'BF': 'Burkina Faso',
 'BI': 'Burundi',
 'KH': 'Cambodia',
 'CM': 'Cameroon',
 'CA': 'Canada',
 'CV': 'Cape Verde',
 'KY': 'Cayman Islands',
 'CF': 'Central African Republic',
 'TD': 'Chad',
 'CL': 'Chile',
 'CN': 'China',
 'CX'

In [32]:
colors = {}
euCountries = data["reporter"].unique()

random.seed(2) # Seed to guarantee the same colours every time
for i in range(len(euCountries)): # Generate random colours for each EU country node, so their edges can be coloured too
    colors[euCountries[i]] = "#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) 
colors

{'AT': '#122B59',
 'BE': '#8615DC',
 'BG': '#BE810B',
 'CY': '#EACD55',
 'CZ': '#7705A5',
 'DE': '#4B5EDB',
 'DK': '#BBE5CE',
 'EE': '#7F8FBE',
 'ES': '#EBEF7A',
 'FI': '#58F99D',
 'FR': '#96FB2A',
 'GR': '#063118',
 'HR': '#734876',
 'HU': '#1D11BB',
 'IE': '#570232',
 'IT': '#010B84',
 'LT': '#550C17',
 'LU': '#410B39',
 'LV': '#AF09E1',
 'MT': '#8C4F72',
 'NL': '#A30E4C',
 'PL': '#FA4A88',
 'PT': '#D04181',
 'RO': '#4553E7',
 'SE': '#177E28',
 'SI': '#27B8D8',
 'SK': '#041CD5'}

In [41]:
def formatValue(num): # Format monetary values into human-readable format with millions and billions
    if num > 1000000000:
        if not num % 1000000000:
            return f'€{num // 1000000000}B'
        return f'€{round(num / 1000000000, 1)}B'        
    elif num > 1000000:
        if not num % 1000000:
            return f'€{num // 1000000}M'
        return f'€{round(num / 1000000, 1)}M'
    return f'€{num // 1000}K'

In [42]:
formatValue(1234567891)

'€1.2B'

In [44]:
visnet = net.Network(notebook = True, cdn_resources = "in_line", directed = True, select_menu = True, filter_menu = True) # Create network (note that the parameters used are for displaying it in-line in a Jupyter notebook; if you use something else, these params might be different)

for idx, row in filteredData.iterrows():
  
    reporterImage = "https://flagicons.lipis.dev/flags/4x3/" + row['reporter'].lower() + ".svg" # Country icon URLs
    partnerImage = "https://flagicons.lipis.dev/flags/4x3/" + row['partner'].lower() + ".svg"
    
    reporterDegree = filteredData[filteredData["reporter"] == row["reporter"]]["reporter"].count() # Country degrees, which determine the size of the node
    partnerDegree = filteredData[filteredData["partner"] == row["partner"]]["partner"].count()

    visnet.add_node(n_id = row['reporter'], label = countries_by_code[row['reporter']], title = row['reporter'], image = reporterImage, shape = 'circularImage', size = int(reporterDegree) * 2)
    visnet.add_node(n_id = row['partner'], label = countries_by_code[row['partner']], title = row['partner'], image = partnerImage, shape = 'circularImage', size = int(partnerDegree) * 2)
    visnet.add_edge(source = row['reporter'], to = row['partner'], value = int(row['OBS_VALUE']) / 200000000, title = formatValue(int(row['OBS_VALUE'])), color = colors[row['reporter']])

visnet.repulsion(spring_length = 500, node_distance = 300) # Spread the nodes out so that the graph is easier to process
visnet.show_buttons(filter_ = ["physics"]) # Show physics control panel
visnet.save_graph("network-2022-test.html") # Display graph

gio: file:///mnt/c/Users/Mate/Documents/ELTE%20Coursework/Fall%202023/Network%20Science/eu-car-trade-network/network-2022-test.html: No application is registered as handling this file


In [53]:
country_to_graph = "Germany"
one_country_data = filtered_data[filtered_data['reporter'] == countries[country_to_graph]]
visnet = net.Network(notebook = True, cdn_resources = "in_line", directed = True, select_menu = True, filter_menu = True) # Create network (note that the parameters used are for displaying it in-line in a Jupyter notebook; if you use something else, these params might be different)

for idx, row in one_country_data.iterrows():
    
    reporterImage = "https://flagicons.lipis.dev/flags/4x3/" + row['reporter'].lower() + ".svg" # Country icon URLs
    partnerImage = "https://flagicons.lipis.dev/flags/4x3/" + row['partner'].lower() + ".svg"
    
    reporterDegree = one_country_data[one_country_data["reporter"] == row["reporter"]]["reporter"].count() # Country degrees, which determine the size of the node
    partnerDegree = one_country_data[one_country_data["partner"] == row["partner"]]["partner"].count()

    visnet.add_node(n_id = row['reporter'], label = countries_by_code[row['reporter']], title = row['reporter'], image = reporterImage, shape = 'circularImage', size = int(reporterDegree) * 2)
    visnet.add_node(n_id = row['partner'], label = countries_by_code[row['partner']], title = row['partner'], image = partnerImage, shape = 'circularImage')
    visnet.add_edge(source = row['reporter'], to = row['partner'], value = int(row['OBS_VALUE']) / 200000000, title = formatValue(int(row['OBS_VALUE'])), color = colors[row['reporter']])

visnet.repulsion(spring_length = 500, node_distance = 300) # Spread the nodes out so that the graph is easier to process
visnet.show_buttons(filter_ = ["physics"]) # Show physics control panel
visnet.show("germany-graph.html") # Display graph